# Домашнее задание 2

In [26]:
# Загрузка необходимых библиотек
!pip install --quiet optuna
import optuna
import pandas as pd
import numpy as np
import lightgbm as lgb
from google.colab import files
from sklearn.metrics import make_scorer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error

In [27]:
# Скачивание файлов
print("Загрузите файл train.csv")
uploaded_train = files.upload()
train_data_main = pd.read_csv(list(uploaded_train.keys())[0])

print("Загрузите файл test.csv")
uploaded_test = files.upload()
test_data_main = pd.read_csv(list(uploaded_test.keys())[0])

Загрузите файл train.csv


Saving train.csv to train (5).csv
Загрузите файл test.csv


Saving test.csv to test (5).csv


In [28]:
# Основной код
train = train_data_main
test = test_data_main

# Разделение данных на признаки и целевую переменную
X = train.drop(['index', 'y'], axis=1)
X_test = test.drop(['index'], axis=1)
y = train['y']

# Разделение на обучающую и валидационную выборки
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.15, random_state=42)

# Кастомные функции
def mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / np.maximum(1e-6, np.abs(y_true)))) * 100

def mape_loss(y_pred, data):
    y_true = data.get_label()
    grad = (y_pred - y_true) / np.maximum(1e-6, np.abs(y_true))
    hess = 1 / np.maximum(1e-6, np.abs(y_true))
    return grad, hess

def mape_metric(y_pred, data):
    y_true = data.get_label()
    mape = np.mean(np.abs((y_true - y_pred) / np.maximum(1e-6, np.abs(y_true))))
    return 'mape', mape, False

def objective(trial):
    param = {
        # 'n_estimators': trial.suggest_int('n_estimators', 500, 1000),
        # 'max_depth': trial.suggest_int('max_depth', 4, 8),
        # 'min_child_samples': trial.suggest_int('min_child_samples', 1, 50),
        # 'subsample': trial.suggest_float('subsample', 0.8, 1.0),
        # 'colsample_bytree': trial.suggest_float('colsample_bytree', 0.8, 1.0),
        # 'max_bin': trial.suggest_int('max_bin', 512, 1024),
        # 'lambda_l1': trial.suggest_float('lambda_l1', 1e-4, 1.0),
        # 'lambda_l2': trial.suggest_float('lambda_l2', 1e-4, 1.0),
        'objective': mape_loss,
        'metric': 'None',
        'verbosity': -1,
        'boosting_type': 'goss',
        'learning_rate': trial.suggest_float('learning_rate', 1e-4, 1e-1, log = True),
        'num_leaves': trial.suggest_int('num_leaves', 20, 70),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 120),
        'random_state': trial.suggest_int('random_state', 1, 50),
    }

    # Создание датасетов для LightGBM
    train_data = lgb.Dataset(X_train, label=y_train)
    valid_data = lgb.Dataset(X_val, label=y_val, reference=train_data)

    # Обучение модели
    gbm = lgb.train(
        param,
        train_data,
        valid_sets=[valid_data],
        num_boost_round=1000,
        feval = mape_metric,
        callbacks=[lgb.early_stopping(stopping_rounds=30)]
    )

    # Предсказание и вычисление MAPE
    y_pred = gbm.predict(X_val, num_iteration=gbm.best_iteration)
    mape_score = mape(y_val, y_pred)

    return mape_score

# Создание и запуск исследования
# study = optuna.create_study(direction='minimize')
# study.optimize(objective, n_trials=1000)

# print("Лучшие гиперпараметры:")
# print(study.best_params)
# print(f"Лучший MAPE: {study.best_value:.4f}")

# Обучение модели с лучшими гиперпараметрами
# best_params = study.best_params
# best_params['objective'] = mape_loss
# best_params['metric'] = 'None'
# best_params['verbosity'] = -1
# best_params['boosting_type'] = 'goss'

# Эти гиперпараметры показали самый высокий результат
best_params = {
        'objective': mape_loss,
        'metric': 'None',
        'verbosity': -1,
        'boosting_type': 'goss',
        'learning_rate': 0.007720473133109053,
        'num_leaves': 31,
        'min_data_in_leaf': 10,
        'random_state': 27,
    }

train_data = lgb.Dataset(X_train, label=y_train)
valid_data = lgb.Dataset(X_val, label=y_val, reference=train_data)

print("\nОбучение модели LightGBM с лучшими гиперпараметрами...")
gbm = lgb.train(
    best_params,
    train_data,
    valid_sets=[valid_data],
    num_boost_round=1000,
    feval = mape_metric,
    callbacks=[lgb.early_stopping(stopping_rounds=30)]
)

# Оценка модели
y_pred_val = gbm.predict(X_val, num_iteration=gbm.best_iteration)
mape_final = mean_absolute_percentage_error(y_val, y_pred_val)
print(f"MAPE на валидационной выборке: {mape_final:.4f}")

# Предсказание на тестовой выборке
y_test_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)

# Создание файла для отправки
submission = pd.DataFrame({
    'index': test['index'],
    'y': y_test_pred
})

submission.to_csv('submission.csv', index=False)
print("Файл submission.csv успешно создан.")

# Загрузка файла на локальный компьютер
files.download('submission.csv')


Обучение модели LightGBM с лучшими гиперпараметрами...
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[274]	valid_0's mape: 0.241137
MAPE на валидационной выборке: 0.2411
Файл submission.csv успешно создан.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>